In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import h5py
import boto.s3
import glob

#### SF 2018

In [5]:
# File location on S3 (The address should be updated depending on the version of the code using)
loc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220528/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220529//beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220529//beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz025 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.25-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.5-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz150 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.5-20220328/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/beam/year-2018-iteration-5/ITERS/it.0/"

#### SF 2019

In [6]:
loc_2019_transCap05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_capacity_0.5-20220329/beam/year-2019-iteration-5/ITERS/it.0/"

In [7]:
%%time
# Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object",
    'primaryFuelType': "category",
    'secondaryFuelType': 'category',
    'currentTourMode': 'category',
    'currentActivity': 'category',
    'nextActivity': 'category'    
}
# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(loc_2018_transitFreq1p5 + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:24: DtypeWarning: Columns (2,8,14,23,43,47,52,56,57) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 3min 10s
Wall time: 3min 53s


In [8]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None 

In [9]:
eventsSF['scenario'] = "TransitFreq1p5" 
eventsSF['scenario'] = eventsSF['scenario'].astype("category")
eventsSF['year'] = 2018

In [10]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [11]:
eventsSFCopy = eventsSF.copy()

In [12]:
# deadhead = eventsSF[eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)
# When there are no riders. Driver and person are not numbers, but include "Agent"

In [14]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [15]:
# shift column 'person' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [16]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [17]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [18]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [19]:
# Shift column 'IDMerged' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [20]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

CPU times: total: 1min 30s
Wall time: 1min 30s


In [21]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [22]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [23]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [24]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(3474, 65)

In [25]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

CPU times: total: 4min 33s
Wall time: 4min 33s


In [26]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [27]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(0, 66)

In [28]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [29]:
print(eventsSF.currentTourMode.unique().tolist())

['hov2_teleportation', nan, 'walk', 'car', 'car_hov3', 'hov3_teleportation', 'bike', 'car_hov2', 'walk_transit', 'drive_transit', 'ride_hail', 'ride_hail_pooled', 'ride_hail_transit', 'bike_transit']


In [30]:
print(eventsSF.modeBEAM.unique().tolist())

['hov2_teleportation', nan, 'walk', 'car', 'car_hov3', 'hov3_teleportation', 'bike', 'car_hov2', 'walk_transit', 'drive_transit', 'ride_hail', 'ride_hail_pooled', 'ride_hail_transit', 'bus', 'tram', 'subway', 'rail', 'cable_car', 'bike_transit']


In [31]:
print(eventsSF.type.unique().tolist())

['ModeChoice', 'actend', 'departure', 'TeleportationEvent', 'arrival', 'actstart', 'PersonEntersVehicle', 'PathTraversal', 'PersonLeavesVehicle', 'LeavingParkingEvent', 'ParkingEvent', 'PersonCost', 'ReserveRideHail', 'Replanning', 'ChargingPlugInEvent', 'RefuelSessionEvent', 'ChargingPlugOutEvent']


In [32]:
%%time
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

CPU times: total: 17.1 s
Wall time: 17.1 s


In [33]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [34]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail' , eventsSF['modeBEAM'])

In [35]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [36]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [37]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [38]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [39]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [40]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'] != 'nan'), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [41]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'] != 'nan'), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [42]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

#### Adding census blocks

In [43]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'):
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]

In [44]:
BGs = gpd.read_file('C:/Shared-Work/Data/Scenario_2010_shp/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')

In [45]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'startX', 'startY', 'BlockGroupStart')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: total: 59min 55s
Wall time: 59min 54s


In [46]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'endX', 'endY', 'BlockGroupEnd')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: total: 1h
Wall time: 1h


#### Adding new columns

In [47]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [48]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [49]:
eventsSF['duration_travelling'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [50]:
eventsSF['distance_travelling'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [51]:
eventsSF['distance_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [52]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['duration_travelling'], np.nan)

In [53]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['distance_travelling'], np.nan)

In [54]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['duration_travelling'], np.nan)

In [55]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['distance_travelling'], np.nan)

In [56]:
eventsSF['duration_in_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), 
                                            eventsSF['duration_travelling'], np.nan)

In [57]:
eventsSF['distance_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [58]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['duration_travelling'], np.nan)

In [101]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['distance_travelling'], np.nan)

In [102]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['duration_travelling'], np.nan)

In [103]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [104]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [105]:
#eventsSF = eventsSF.set_index('IDMerged')

In [106]:
eventsSF.modeBEAM_rh.unique()

array(['hov3_teleportation', nan, 'car', 'walk', 'ride_hail', 'car_hov2',
       'walk_transit', 'bus', 'cable_car', 'subway', 'hov2_teleportation',
       'rail', 'tram', 'bike', 'car_hov3', 'ride_hail_transit',
       'drive_transit', 'ride_hail_pooled', 'bike_transit'], dtype=object)

In [107]:
eventsSF.primaryFuelType.unique()

[NaN, 'Food', 'Gasoline', 'Diesel', 'Electricity', 'Biodiesel']
Categories (5, object): ['Gasoline', 'Biodiesel', 'Diesel', 'Electricity', 'Food']

In [108]:
eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [109]:
eventsSF['emissionFood'] = eventsSF['fuelFood'] * 8.3141841e-9 * 0

In [110]:
eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                eventsSF['primaryFuel'], np.nan)

In [111]:
eventsSF['emissionElectricity'] = eventsSF['fuelElectricity'] * 2.77778e-10 * 947.2 * 0.0005

In [112]:
eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [113]:
eventsSF['emissionDiesel'] = eventsSF['fuelDiesel'] * 8.3141841e-9 * 10.180e-3

In [114]:
eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [115]:
eventsSF['emissionBiodiesel'] = eventsSF['fuelBiodiesel'] * 8.3141841e-9 * 10.180e-3

In [116]:
eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            , eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [117]:
eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [118]:
eventsSF['emissionGasoline'] = eventsSF['fuelGasoline'] * 8.3141841e-9 * 8.89e-3

In [119]:
# Marginal fuel
conditions  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled'), 
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),
               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')]
choices = [eventsSF['fuel_not_Food']/eventsSF['numPassengers'], 0 , eventsSF['fuelFood'], eventsSF['fuel_not_Food']]

In [120]:
eventsSF['fuel_marginal'] = np.select(conditions, choices, default=np.nan)

In [121]:
#eventsSF[(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelElectricity'].notna() != 0)].head()

In [122]:
# Marginal emission
conditions1  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelElectricity'].notna() != 0), 
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelGasoline'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelBiodiesel'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelDiesel'].notna() != 0),             
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),

               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelElectricity'].notna() != 0),
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelGasoline'].notna() != 0),           
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelBiodiesel'].notna() != 0),   
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelDiesel'].notna() != 0),

               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelFood'].notna() != 0)]

choices1 = [eventsSF['emissionElectricity']/eventsSF['numPassengers'],
           eventsSF['emissionGasoline']/eventsSF['numPassengers'],
           eventsSF['emissionBiodiesel']/eventsSF['numPassengers'],
           eventsSF['emissionDiesel']/eventsSF['numPassengers'],           
           0 , 
           eventsSF['emissionFood'], 
           eventsSF['emissionElectricity'],
           eventsSF['emissionGasoline'],
           eventsSF['emissionBiodiesel'],
           eventsSF['emissionDiesel'],
           eventsSF['emissionFood']]

In [123]:
eventsSF['emission_marginal'] = np.select(conditions1, choices1, default=np.nan)

In [124]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [125]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

#### Trip Index

In [126]:
#eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripId.fillna(method='ffill')

#### Previous code for mode choice 

In [127]:
# Make a new column which determined mode choice numbers for each person and trip
#s = eventsSF.groupby(['IDMerged','tripIndex', 'type']).cumcount().add(1).astype(str).str.zfill(2)

In [128]:
#eventsSF['type_number'] = eventsSF['type'].astype(str) + s.astype(str)

In [129]:
#eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice01' , eventsSF['modeBEAM_rh'], np.nan)

In [ ]:
# Just showing the mode choice after replanning
#eventsSF['mode_choice_replanned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice02', eventsSF['modeBEAM_rh'], np.nan)

#### Only works if we have one replanning event

In [ ]:
# Showing both mode choices before and after the replanning event
#eventsSF['mode_choice_act'] = eventsSF['mode_choice_replanned_BEAM'].combine_first(eventsSF['mode_choice_planned_BEAM'])

In [ ]:
# If replanned, then the status is 1
#eventsSF['replanning_status'] = np.where(eventsSF['mode_choice_replanned_BEAM'].notna() , 1, np.nan)

In [ ]:
# Group by and fill all the rows related to the replanning event
#eventsSF["replanning_status"] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False)['replanning_status'].transform(lambda x: x.ffill().bfill())

In [100]:
# Put all non-replanning events equal to 0
#eventsSF["replanning_status"] = np.where(eventsSF['replanning_status'].isna(), 0, eventsSF['replanning_status']).astype(int)

In [101]:
#eventsSF['mode_choice_actual_BEAM'] = np.where((eventsSF['replanning_status'] == 1) & (eventsSF['type_number']=='ModeChoice01'), np.nan, eventsSF['mode_choice_act'])

In [102]:
#eventsSF = eventsSF.drop(['mode_choice_act'], axis = 1)

In [103]:
#eventsSF[eventsSF['type'] == "Replanning"].tail(16)

In [104]:
#num_pass.to_csv('C:/Shared-Work/Data/CleanData/num_pass.csv', index = False)

In [105]:
#eventsSF["mode_choice_replanned_BEAM"] = eventsSF.groupby(['IDMerged','tripIndex'])['mode_choice_replanned_BEAM'].transform(lambda x: x.bfill())

In [106]:
#eventsSF['mode_choice_actual_BEAM'] = eventsSF['mode_choice_actual_BEAM'].combine_first(eventsSF['modeBEAM_rh'])

In [107]:
#eventsSF['mode_planned_tour_ActSim'] = np.where(eventsSF['type'] == 'PathTraversal', eventsSF['currentTourMode'], np.nan)

In [108]:
#eventsSF.head(10000).to_csv('C:/Shared-Work/Data/CleanData/eventsNew_check.csv', index = False)   

In [109]:
#eventsSF['numPassengers_agg'] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False).numPassengers.agg(lambda x: ', '.join(set(x.dropna().astype(str))))

#### Mode Choice planned and actual

In [130]:
%%time
eventsSF['mode_choice_actual_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('last')

CPU times: total: 1min 28s
Wall time: 1min 28s


In [131]:
%%time
eventsSF['mode_choice_planned_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('first')

CPU times: total: 1min 28s
Wall time: 1min 28s


In [132]:
eventsSF['mode_choice_actual_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_actual_BEAM'])

In [133]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_planned_BEAM'])

In [134]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [135]:
#eventsSF[eventsSF['type']=='Replanning'].head()

In [136]:
# Replanning events = 1, the rest = 0
eventsSF['replanning_status'] = np.where(eventsSF['type']=='Replanning', 1, 0)

In [137]:
#q = eventsSF[eventsSF['IDMerged'] == 6902751]

In [138]:
#q.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/fake2.csv')   

In [139]:
%%time
eventsSF['reason'].replace('nan', np.NaN)

CPU times: total: 1.86 s
Wall time: 1.86 s


In [140]:
eventsSF['transit_bus'] = np.where(eventsSF['modeBEAM_rh']=='bus', 1, 0)
eventsSF['transit_subway'] = np.where(eventsSF['modeBEAM_rh']=='subway', 1, 0)
eventsSF['transit_tram'] = np.where(eventsSF['modeBEAM_rh']=='tram', 1, 0)
eventsSF['transit_rail'] = np.where(eventsSF['modeBEAM_rh']=='rail', 1, 0)
eventsSF['transit_cable_car'] = np.where(eventsSF['modeBEAM_rh']=='cable_car', 1, 0)

In [141]:
eventsSF['ride_hail_pooled'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail_pooled', 1, 0)

In [ ]:
#transit_data = eventsSF[(eventsSF['modeBEAM_rh'] == "bus") | (eventsSF['modeBEAM_rh'] == "subway") 
#                        | (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'ride_hail_transit') 
#                        | (eventsSF['modeBEAM_rh'] == 'bike_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit') 
#                        | (eventsSF['modeBEAM_rh'] == 'cable_car') | (eventsSF['modeBEAM_rh'] == 'rail') 
#                        | (eventsSF['modeBEAM_rh'] == 'tram')]

In [ ]:
#eventsSF.loc['Total',:]= df.sum(axis=0)

In [124]:
#eventsSF.type[eventsSF.type == 'Replanning'].count()

In [125]:
#Person_Trip_eventsSF.replanning_status[Person_Trip_eventsSF.replanning_status == 0].count()

In [126]:
#rh_transit5 = eventsSF[eventsSF['IDMerged']== 3671596]

In [127]:
#rh_transit5.to_csv('C:/Shared-Work/Data/CleanData/rh_transit5.csv', index = False)

In [128]:
eventsSFCopy = eventsSF.copy()

In [129]:
eventsSF.head(200000).to_csv('C:/Shared-Work/Data/CleanData/modeBEAMCHECK.csv', index = False)

In [142]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'duration_travelling': np.sum, 'cost_BEAM': np.sum, 'actStartType': np.sum, 
            'actEndType': np.sum, 'duration_walking': np.sum, 'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 'distance_travelling': np.sum, 'duration_in_transit': np.sum, 'distance_walking': np.sum, 
            'distance_bike': np.sum, 'distance_ridehail': np.sum, 'distance_privateCar': np.sum, 'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 
            'mode_choice_planned_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'vehicle': lambda x: ', '.join(set(x.dropna().astype(str))),
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'reason': lambda x: ', '.join(list(x.dropna().astype(str))),
            'parkingType': lambda x: ', '.join(list(x.dropna().astype(str))),
            'transit_bus': np.sum, 'transit_subway': np.sum, 'transit_tram': np.sum, 'transit_cable_car': np.sum,
            'ride_hail_pooled': np.sum, 'transit_rail': np.sum,
            'year': lambda x: ', '.join(set(x.dropna().astype(str))),
            'scenario': lambda x: ', '.join(set(x.dropna().astype(str))),
            'fuelFood': np.sum, 'fuelElectricity': np.sum, 'fuelBiodiesel': np.sum, 
            'fuelDiesel': np.sum, 'fuel_not_Food': np.sum, 'fuelGasoline': np.sum, 'fuel_marginal': np.sum,
            'BlockGroupStart': 'first',
            'BlockGroupEnd': 'last',
            'emissionFood': np.sum, 'emissionElectricity': np.sum, 'emissionDiesel': np.sum, 'emissionGasoline': np.sum,
            'emissionBiodiesel': np.sum, 'emission_marginal': np.sum
           }).reset_index() 

#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str))) 
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))) #
#'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 

CPU times: total: 39min 28s
Wall time: 39min 28s


In [143]:
Person_Trip_eventsSF['duration_door_to_door'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [144]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['duration_door_to_door'] - Person_Trip_eventsSF['duration_travelling'] 

In [145]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [ ]:
#Person_Trip_eventsSF[Person_Trip_eventsSF['IDMerged'] == 2192]

In [ ]:
#Person_Trip_eventsSF.rename(columns={"modeBEAM_rh":"mode_BEAM_list"}, inplace=True) 

In [146]:
Person_Trip_eventsSF.rename(columns={"legVehicleIds":"vehicleIds_estimate"}, inplace=True) 

In [147]:
Person_Trip_eventsSF.rename(columns={"vehicle":"vehicleIds"}, inplace=True) 

In [148]:
#Person_Trip_eventsSF.door_to_door_time[Person_Trip_eventsSF.door_to_door_time<0].count()

In [ ]:
#negative_door = Person_Trip_eventsSF[(Person_Trip_eventsSF.door_to_door_time<0)]

In [149]:
# Column with five summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car']

In [150]:
Person_Trip_eventsSF['mode_choice_actual_5'] = np.select(conditions, choices, default=np.nan)

In [ ]:
#negative_door.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/negative_door.csv')   

In [ ]:
Person_Trip_eventsSF = Person_Trip_eventsSF.drop(Person_Trip_eventsSF[Person_Trip_eventsSF.duration_door_to_door < 0].index)

In [151]:
Person_Trip_eventsSF.to_csv('C:/Shared-Work/Data/CleanData/sf_2018_tr_fr_1p5_BEAM.csv', index = False)

In [146]:
#Person_Trip_eventsSF[Person_Trip_eventsSF['mode_choice_planned_BEAM'] == 'walk_transit'].head()

#### Merging with activity sim persons and housholds files

In [152]:
actloc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/activitysim/"
actloc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220528/activitysim/"
actloc_2018_transitFreq1p5 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_1.5-20220529//activitysim/"
actloc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220529//activitysim/"
actloc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/activitysim/"
actloc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/activitysim/"

households = pd.read_csv(actloc_2018_transitFreq1p5 + 'final_households.csv')
persons = pd.read_csv(actloc_2018_transitFreq1p5 + 'final_persons.csv')
tours = pd.read_csv(actloc_2018_transitFreq1p5 +'final_tours.csv')
plans = pd.read_csv(actloc_2018_transitFreq1p5 +'final_plans.csv')
trips = pd.read_csv(actloc_2018_transitFreq1p5 + 'final_trips.csv')

In [153]:
# PUMS data for Disability information
filename = "C:/Shared-Work/Data/Disability_PUMS/custom_mpo_06197001_model_data.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data = list(f[a_group_key])

Keys: <KeysViewHDF5 ['blocks', 'counties', 'ect', 'edges', 'hct', 'household_targets_acs', 'household_validation_acs', 'households', 'job_county_targets_BEA', 'job_county_validation_BEA', 'job_flows', 'job_targets', 'job_validation', 'jobs', 'nodes', 'persons', 'price_targets', 'price_validation', 'residential_units', 'travel_data', 'unit_county_targets_PEP', 'unit_county_validation_PEP', 'unit_targets', 'unit_validation', 'validation_ect', 'validation_hct', 'values']>


In [154]:
# PUMS data for Disability information
person_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/ss13pca.csv')
housing_unit_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_hca/ss13hca.csv')

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_384\1998398585.py:2: DtypeWarning: Columns (107,108,109) have mixed types. Specify dtype option on import or set low_memory=False.
  person_5_year_2010 = pd.read_csv('C:/Users/nazanin/Downloads/csv_pca/ss13pca.csv')


In [155]:
# Merge PUMS households and persons 
person_5_year_2010 = person_5_year_2010.sort_values(by=['SERIALNO']).reset_index()
housing_unit_5_year_2010 = housing_unit_5_year_2010.sort_values(by=['SERIALNO']).reset_index()
hhpersons_PUMS2010 = pd.merge(left=person_5_year_2010, right=housing_unit_5_year_2010, how='left', on='SERIALNO')

In [156]:
# Merge households and persons 
persons = persons.sort_values(by=['household_id']).reset_index(drop=True)
households = households.sort_values(by=['household_id']).reset_index(drop=True)
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [157]:
hhpersons.head()

,person_id,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
0,1140762,115000.0,1,0,0,female,1,2,14,50.0,2,above 60,24.0,-1,1,white,0,63,1406,no,0,0,1005,1,1,3,-122.225696,37.857091,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,4.519702,-1,NaN,0.0,1262,13.876887,0.004654,True,1.0,135.266660,613.80084,1.0,True,N,True,False,False,False,NaN,False,False,0,0,0,20,2,0,1,1,0,0,0,2,1,0.0,1,6001,no,60014001001046,0,no,63,1,own not recent,one,yes,1,one,118500.0,gt35-lt65,2010000113222,1.0,white,gt100-lt150,0,one,0,no,1,2,1005,1,0.097,126428,118.5,4,12.86,4.519702,0.0,1,1,0,0,0,0,0,0,False,True,False,False,1,1.0,0,1,1,0,0,1,False,0_tours,0
1,1336452,82000.0,1,0,0,female,1,2,16,37.0,5,above 60,22.0,-1,1,white,0,63,401,no,0,0,1005,1,1,3,-122.232831,37.856070,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,7.220078,-1,NaN,0.0,401,13.937266,0.002449,False,3.0,1.091662,775.94390,1.0,False,N,True,False,False,False,NaN,False,False,0,0,0,20,3,0,1,2,0,0,0,3,1,0.0,1,6001,no,60014001001049,0,no,63,1,own not recent,one,yes,1,one,129000.0,gt35-lt65,2013000127260,1.0,white,gt100-lt150,0,one,0,no,1,2,1005,1,0.097,15051,129.0,4,12.86,7.220078,0.0,1,1,0,0,0,0,0,0,False,True,False,False,1,1.0,0,1,1,0,0,1,False,0_tours,0
2,1956965,190000.0,1,0,0,female,1,2,24,50.0,5,35 to 60,23.0,-1,6,asian,0,57,401,no,0,0,1005,1,1,3,-122.226436,37.855050,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0,True,1005,12.171989,-1,NaN,0.0,401,14.117073,0.002449,False,3.0,1.091662,775.94390,1.0,False,M,True,False,False,False,work1,False,False,1,1,0,0,0,0,0,0,0,0,0,0,1,0.0,1,6001,no,60014001001054,0,no,57,6,own not recent,one,no,1,one,196000.0,gt35-lt65,2013000674070,1.0,asian,gt150,0,one,0,no,1,2,1005,1,0.097,208708,196.0,4,12.86,12.171989,0.0,1,1,0,0,0,0,0,0,False,True,False,False,1,1.0,0,1,1,0,0,1,False,0_tours,0
3,5171467,0.0,0,0,0,female,1,2,65,0.0,

In [158]:
# Merge Disability Columns
hhpersons_PUMS2010 = hhpersons_PUMS2010.sort_values(by=['SERIALNO', 'SPORDER']).reset_index(drop=True)
hhpersons = hhpersons.sort_values(by=['serialno', 'PNUM']).reset_index(drop=True)
hhpersonsDIS = pd.merge(hhpersons, hhpersons_PUMS2010[['SERIALNO', 'SPORDER', 'AGEP', 'SEX', 'DIS', 'HINCP', 'VEH', 'JWMNP', 'JWRIP', 'JWTR', 'RAC1P', 'RAC2P05']], how='left',left_on = ['serialno', 'PNUM'] , right_on=['SERIALNO', 'SPORDER'])

In [159]:
hhpersons[hhpersons['serialno']==2009000000022]

,person_id,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
0,2498343,100000.0,1,0,0,male,1,1,2428308,40.0,3,35 to 60,21.0,-1,1,white,0,58,1393,no,0,0,1375,1,1,3,-122.762097,38.439462,False,True,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0,True,1375,9.894720,-1,NaN,0.000000,1387,12.472154,13.867542,False,5.0,72.700000,382.607360,1.000000,False,M,True,False,False,False,work1,False,False,1,1,0,0,0,0,0,0,0,0,0,0,1,0.0,2,6097,no,60971530062004,1,no,58,1,own not recent,two,yes,1,two or more,150000.0,gt35-lt65,2009000000022,1.0,white,gt150,0,one,0,no,2,4,1375,4,0.097,198632,150.0,4,12.86,9.894720,1.0,2,2,0,0,0,0,0,1,True,False,False,False,2,1.000000,0,2,2,0,0,2,True,0_tours,0
1,1988405,100000.0,1,0,0,male,1,1,2438428,40.0,3,35 to 60,21.0,-1,1,white,0,58,1377,no,0,0,1381,1,1,3,-122.754259,38.375504,False,True,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0,True,1381,4.323352,-1,NaN,0.000000,1380,12.673839,3.387133,False,4.0,29.740715,88.709274,0.739244,False,M,True,False,False,False,work1,False,False,1,1,0,0,0,0,0,0,0,0,0,0,1,0.0,2,6097,no,60971533003040,1,no,58,1,own not recent,two,yes,1,two or more,150000.0,gt35-lt65,2009000000022,1.0,white,gt150,0,one,0,no,2,4,1381,4,0.097,261004,150.0,4,12.86,4.323352,1.0,2,2,0,0,0,0,0,1,True,False,False,False,2,0.739244,0,2,2,0,0,2,True,0_tours,0
2,4135595,100000.0,1,0,0,male,1,1,2531272,40.0,3,35 to 60,21.0,-1,1,white,0,58,1407,no,0,0,1350,1,1,3,-122.678266,38.416753,False,True,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0,True,1350,9.827892,-1,NaN,0.000000,1409,12.590604,28.669370,False,4.0,70.000000,1039.991100,1.000000,False,N,True,False,False,False,NaN,False,False,0,0,0,18,2,0,0,1,0,0,1,2,1,0.0,2,6097,no,60971515022004,1,no,58,1,own not recent,two,yes,1,two or more,150000.0,gt35-lt65,2009000000022,1.0,white,gt150,0,one,0,no,2,4,1350,4,0.097,135421,150.0,4,12.86,9.827892,1.0,2,2,0,0,0,0,0,1,True,False,False,False,2,

In [160]:
# Merge tours, households and persons
tours = tours.sort_values(by=['person_id']).reset_index(drop=True)
hhpersonsDIS = hhpersonsDIS.sort_values(by=['person_id']).reset_index(drop=True)
hhperTours = pd.merge(left=tours, right=hhpersonsDIS, how='left', on='person_id')
#hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
#hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [161]:
# Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id']).reset_index(drop=True)
hhperTours = hhperTours.sort_values(by=['person_id','tour_id']).reset_index(drop=True)
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'])
#tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
#tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [162]:
# Merge trips, tours, households, and persons with land-use
#landuse = landuse.sort_values(by=['TAZ'])
#tourTripsMerged = tourTripsMerged.sort_values(by=['TAZ'])
#landuseTripsMerged = pd.merge(tourTripsMerged, landuse, how='left', left_on='TAZ', right_on='TAZ', suffixes=('', '_drop'))
#landuseTripsMerged.drop([col for col in landuseTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [164]:
# Concat mode_choice_raw files
path = "C:/Users/nazanin/Downloads/trip_mode_choice_tr_fre_1p5/trip_mode_choice/"
all_files = glob.glob(path + "*raw.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_raw = pd.concat(li2, axis = 0, ignore_index=True)

In [165]:
# Concat mode_choice_utilities files
path = "C:/Users/nazanin/Downloads/trip_mode_choice_tr_fre_1p5/trip_mode_choice/"
all_files = glob.glob(path + "*utilities.csv")
li_mapper = map(lambda filename: pd.read_csv(filename, index_col = None, header = 0), all_files)
li2 = list(li_mapper)
SFmode_choice_utilities = pd.concat(li2, axis = 0, ignore_index=True)

In [166]:
# Merge mode_choice_raw and utilities
SFmode_choice_raw = SFmode_choice_raw.sort_values(by=['trip_id'])
SFmode_choice_utilities = SFmode_choice_utilities.sort_values(by=['trip_id'])
rawUtil = pd.merge(left=SFmode_choice_raw, right=SFmode_choice_utilities, how='left', on='trip_id')

In [167]:
%%time
# Merge trips, tours, households, persons, trip_mode_choice_raw, and utilities
tourTripsMerged = tourTripsMerged.sort_values(by=['trip_id'])
rawUtil = rawUtil.sort_values(by=['trip_id'])
SFActMerged= pd.merge(left=tourTripsMerged, right=rawUtil, how='left', on=['trip_id'])

CPU times: total: 32.5 s
Wall time: 32.5 s


In [168]:
SFActMerged.head()

,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode,mode_choice_logsum_x,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode,mode_choice_logsum_y,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,SERIALNO,SPORDER,AGEP,SEX,DIS,HINCP,VEH,JWMNP,JWRIP,JWTR,RAC1P,RAC2P05,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,trip_purpose,DRIVEALONEFREE,DRIVEALONEPAY,SHARE

In [169]:
# Merge person_trip level BEAM with activity sim merged files
SFActMerged = SFActMerged.sort_values(by=['person_id', 'trip_id']).reset_index(drop=True)
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripIndex']).reset_index(drop=True)
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=SFActMerged, how='left', left_on=["IDMerged", 'tripIndex'], right_on=['person_id', 'trip_id'])
#eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
#eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

In [170]:
eventsASim.rename(columns={"mode_choice_logsum_y":"logsum_tours_mode_AS_tours"}, inplace=True)

In [171]:
eventsASim.rename(columns={"tour_mode":"tour_mode_AS_tours"}, inplace=True)

In [172]:
eventsASim.rename(columns={"mode_choice_logsum_x":"logsum_trip_mode_AS_trips"}, inplace=True)

In [173]:
eventsASim.rename(columns={"trip_mode":"trip_mode_AS_trips"}, inplace=True)

In [ ]:
%%time
eventsASim.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/TransitHead_Frequencies/sf_2018_tr_fr_1p5.csv', index=False)

In [ ]:
eventsASim.head(10000).to_csv('C:/Shared-Work/Data/CleanData/final_tr_fr_1p5_test.csv', index = False)

In [136]:
#eventsASim.to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/sf_2018_tr_fr_0p5.csv', index = False)

In [ ]:
# eventsASim.duration_in_privateCar.sum()

In [141]:
eventsASim.head(50000).to_csv('C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/sf_2018_base_test.csv', index = False)

In [ ]:
#import matplotlib.pyplot as plt
#import seaborn as sns

#plt.figure(figsize=(13, 6))
#sns.heatmap(households, vmax=1, annot=True, linewidths=.5)
#plt.xticks(rotation=30, horizontalalignment='right')
#plt.show()

In [ ]:
#import missingno as msno
#%matplotlib inline
#msno.bar(SFbaseline2018)
#msno.heatmap(persons)

In [ ]:
#from io import StringIO # python3; python2: BytesIO 
#import boto
#import boto3
#
#conn = boto.s3.connect_to_region('us-east-2')
#bucket = conn.get_bucket('beam-core-act')
#csv_buffer = StringIO()
#eventsASim.to_csv(csv_buffer)
#s3_resource = boto3.resource('s3')
#s3_resource.Object(bucket, 'eventsASim.csv').put(Body=csv_buffer.getvalue())